## Imports and Inputs

In [2]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
# import seaborn as sns
from datetime import datetime, timedelta, timezone

from dune_client.client import DuneClient
from config import key

/tmp/ipykernel_640/2591426926.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# import jsons of capital pool, ramm volume and covers from dune api 
dune = DuneClient(key)
covers_list_result = dune.get_latest_result_dataframe(2303588)

In [4]:
# manually input the valuation date as a string in the format 'dd-mm-yyyy'
valn_date_str = '31-03-2024'

In [5]:
# create valuation datetime
valn_date = datetime.strptime(valn_date_str, '%d-%m-%Y').replace(tzinfo=timezone.utc)
# create valuation datetimes for 30 days and 365 days ago
valn_date_minus_30 = valn_date - timedelta(days=30)
valn_date_minus_365 = valn_date - timedelta(days=365)

# create a string for API for 30 days before valuation date
valn_date_minus_30_str = valn_date_minus_30.strftime('%d-%m-%Y')

In [ ]:
# coingecko ETH API for ETH-USD at valuation date
url = 'https://api.coingecko.com/api/v3/coins/ethereum/market_chart'
eth_price_params = {'vs_currency': 'usd', 
                   'days': '365',
                   'interval': 'daily'}
eth_usd_prices = requests.get(url, params=eth_price_params).json()['prices']

In [19]:
price_df = pd.DataFrame(eth_usd_prices, columns = ['Date', 'EthUsd'])
price_df['Date'] = pd.to_datetime(price_df['Date'], utc=True, unit='ms', origin='unix')

In [20]:
price_df

,Date,EthUsd
0,2023-04-19 00:00:00+00:00,2103.278814
1,2023-04-20 00:00:00+00:00,1936.425921
2,2023-04-21 00:00:00+00:00,1943.435598
3,2023-04-22 00:00:00+00:00,1850.149738
4,2023-04-23 00:00:00+00:00,1877.541328
...,...,...
360,2024-04-13 00:00:00+00:00,3245.500593
361,2024-04-14 00:00:00+00:00,3022.005785
362,2024-04-15 00:00:00+00:00,3157.683766
363,2024-04-16 00:00:00+00:00,3101.190736


In [15]:
eth_usd_prices = {}

def eth_usd(date_str):
    
    url = 'https://api.coingecko.com/api/v3/coins/ethereum/history'
    eth_price_params = {'date': date_str}
    eth_usd_value = requests.get(url, params=eth_price_params).json()['market_data']['current_price']['usd']
    
    return eth_usd_value
    
for i in range(31):
    # loop through dates
    date = valn_date - timedelta(days=i)
    # turn datetime into a string for API
    date_str = date.strftime('%d-%m-%Y')
    eth_usd_prices[date_str] = eth_usd(date_str)

eth_usd_prices

KeyError: 'market_data'

In [16]:
requests.get(url, params=eth_price_params).json()

{'status': {'error_code': 429,
  'error_message': "You've exceeded the Rate Limit. Please visit https://www.coingecko.com/en/api/pricing to subscribe to our API plans for higher rate limits."}}

## Monthly Cover Buy Stats

In [12]:
df_covers = covers_list_result

# change timestamp to datetime format and normalise all dates to midnight
df_covers['cover_start_time'] = pd.to_datetime(df_covers['cover_start_time']).dt.floor('D')

# filter for covers that were bought on valn date and in the 30 days before
df_covers_30_days = df_covers[(df_covers['cover_start_time']>=valn_date_minus_30) & (df_covers['cover_start_time']<=valn_date)]

94        0.094522
95        0.077246
96       15.753098
97      100.812827
98      542.452697
          ...     
205      13.371632
206     150.230242
207    2065.190143
208    8498.529644
209       0.009681
Name: premium_native, Length: 116, dtype: float64

In [14]:
# work out cover fees denominated in ETH, allowing for 15% commission
commission_share = 0.15

monthly_cover_fees_usd = df_covers_30_days['premium_dollar'].sum()
monthly_cover_fees_after_commission = monthly_cover_fees_usd * (1 - commission_share)
monthly_commission = monthly_cover_fees_usd * commission_share

monthly_cover_fees_eth = monthly_cover_fees_usd / eth_usd
monthly_cover_fees_eth

39.58797846624159